## Setting up your keys

If you haven't done so already, you could now create API keys for Anthropic and Google in addition to OpenAI.

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
```

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.

In [2]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

import google.generativeai

In [5]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins with {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins with {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins with {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins with sk-proj-
Anthropic API Key not set
Google API Key not set


In [12]:
# Connect to OpenAI, Anthropic or google

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

In [ ]:
# Claude 3.5 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

All three APIs use the same structure we already used, a message for the system and a prompt for the user.

In [7]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [8]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [9]:
# GPT-3.5-Turbo

completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
print(completion.choices[0].message.content)

Why did the data scientist go broke?

Because he used up all his cache!


In [10]:
# GPT-4o-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the statistician?

Because she found him too mean!


In [11]:
# GPT-4o

completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

Why did the data scientist bring a ladder to work?

Because they heard the cloud was high up!


In [ ]:
# Claude 3.5 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

In [ ]:
# Claude 3.5 Sonnet again
# Now let's add in streaming back results

result = claude.messages.stream(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

In [ ]:
# The API for Gemini has a slightly different structure.
# I've heard that on some PCs, this Gemini code causes the Kernel to crash.
# If that happens to you, please skip this cell and use the next cell instead - an alternative approach.

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

In [14]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [15]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

Determining whether a business problem is suitable for a Large Language Model (LLM) solution involves assessing the nature of the problem and the capabilities of LLMs. Here's a guide to help you decide:

### 1. Problem Characteristics

- **Text-Based**: LLMs excel at processing and generating human language. If your problem involves text, such as natural language processing, text generation, or conversational interfaces, it may be suitable for an LLM.
  
- **Language Understanding**: If the problem requires understanding, summarizing, or translating text, LLMs can be highly effective.

- **Creativity and Generation**: For tasks that require generating creative content, such as writing, brainstorming ideas, or creating marketing materials, LLMs can be valuable.

- **Pattern Recognition in Text**: If the problem involves identifying patterns, sentiments, or extracting information from large volumes of text, consider using an LLM.

### 2. Data Availability

- **Quality and Quantity**: LLMs require large datasets to perform effectively. Ensure you have access to high-quality and relevant text data.

- **Structured vs. Unstructured Data**: LLMs are more suitable for unstructured data (like plain text) rather than highly structured data like databases.

### 3. Output Requirements

- **Precision vs. Creativity**: If the task requires high creativity and flexibility, LLMs are suitable. However, for tasks requiring precise, deterministic outputs (like exact numerical calculations), LLMs might not be the best choice.

- **Interpretability**: If the solution needs to be easily interpretable, LLMs may pose challenges due to their complex nature.

### 4. Context and Constraints

- **Real-Time Processing**: Consider if real-time processing is required. LLMs, especially large ones, might have latency issues.

- **Resource Availability**: LLMs can be resource-intensive. Ensure you have the necessary computational resources and infrastructure.

- **Ethical and Privacy Considerations**: Assess if the use of LLMs aligns with ethical guidelines and privacy regulations, especially when handling sensitive data.

### 5. Business Value

- **Scalability**: Determine if an LLM can scale the solution in a way that adds value to your business.

- **Cost-Benefit Analysis**: Evaluate if the potential benefits of implementing an LLM outweigh the costs and complexity.

### 6. Expertise and Support

- **Technical Expertise**: Ensure your team has the necessary expertise to implement and maintain LLM solutions.

- **Vendor Support**: If using a third-party LLM service, assess the level of support and service provided.

### Conclusion

To decide if a business problem is suitable for an LLM solution, carefully evaluate the nature of the problem, the capabilities of LLMs, and the resources available. Consider conducting a small-scale pilot project to test the feasibility and effectiveness of an LLM before full-scale

RemoteProtocolError: peer closed connection without sending complete message body (incomplete chunked read)